In [18]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from notebooks.imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from kilosort import run_kilosort, io
import spikeinterface.extractors as se

### Load Configs

In [20]:
from config import dir_config, main_config, s_probe_path

raw_dir = Path(dir_config.data.raw)
sorting_dir = Path(dir_config.data.sorting)
compiled_dir = Path(dir_config.data.compiled)

session_metadata = pd.read_csv(Path(compiled_dir, "sessions_metadata.csv"), index_col=None)

### Utils functions

In [21]:

def convert_nsx_to_bin(nsx_path, nsx_name, bin_path, bin_name):
    nsx_data, N, c, s, fs, probe_path = None, None, None, None, None, None
    
    nsx_file = Path(nsx_path, f'{nsx_name}.ns5')
    if nsx_file.exists():
        nsx_data = se.read_blackrock(file_path=Path(nsx_path, nsx_name), stream_id='5')   
        filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
        nsx_data, bin_path, data_name=f"{bin_name}.bin", dtype=np.int16,
        chunksize=60000, export_probe=True, probe_name='probe.prb'
        )
        print(f"{N}, {c}, {s}, {fs}, {probe_path}")
    else:
        print(f"File {nsx_file} does not exist")
    return nsx_data, N, c, s, fs, probe_path

## Covert blackrock (NSX) data to bin

In [22]:
for idx, session_id in enumerate(session_metadata['session_id']):
    # Construct the path to the raw and binary file
    nsx_path = Path(raw_dir) / session_id
    nsx_name = session_id
    bin_path = Path(sorting_dir) / session_id
    bin_name = session_id
    bin_file = bin_path / f"{bin_name}.bin"
    
    try:
        # check if raw data exists
        if not nsx_path.exists():
            print(f"Raw data does not exist for: \t {session_id}")
            continue
        
        # Check if the binary file already exists
        if not bin_file.exists():
            print(f"Converting {session_id} to binary")            
            # Call the conversion function
            nsx_data, N, c, s, fs, probe_path = convert_nsx_to_bin(nsx_path=str(nsx_path), nsx_name=nsx_name, bin_path=str(bin_path), bin_name=bin_name)
        else:
            print(f"Bin file exists for: \t {bin_name}")
    except Exception as e:
        print(f"Error: {e}")
        continue

kilosort.io : ========================================
kilosort.io : ========================================
kilosort.io : Loading recording with SpikeInterface...
kilosort.io : Loading recording with SpikeInterface...
kilosort.io : number of samples: 181721280
kilosort.io : number of samples: 181721280
kilosort.io : number of channels: 18
kilosort.io : number of channels: 18
kilosort.io : numbef of segments: 1
kilosort.io : numbef of segments: 1
kilosort.io : sampling rate: 30000.0
kilosort.io : sampling rate: 30000.0
kilosort.io : dtype: int16
kilosort.io : dtype: int16


Bin file exists for: 	 210126_GP_JP
Raw data does not exist for: 	 210205_GP_JP
Raw data does not exist for: 	 210210_GP_JP
Raw data does not exist for: 	 210211_GP_JP
Raw data does not exist for: 	 210216_GP_JP
Raw data does not exist for: 	 210217_GP_JP
Raw data does not exist for: 	 210224_GP_JP
Raw data does not exist for: 	 210302_GP_JP
Raw data does not exist for: 	 210305_GP_JP
Raw data does not exist for: 	 210311_GP_JP
Raw data does not exist for: 	 210312_GP_JP
Raw data does not exist for: 	 210315_GP_JP
Raw data does not exist for: 	 210316_GP_JP
Raw data does not exist for: 	 210317_GP_JP
Raw data does not exist for: 	 210318_GP_JP
Raw data does not exist for: 	 210323_GP_JP
Raw data does not exist for: 	 210413_GP_JP
Raw data does not exist for: 	 210524_GP_JP
Raw data does not exist for: 	 210525_GP_JP
Raw data does not exist for: 	 210528_GP_JP
Raw data does not exist for: 	 210601_GP_JP
Bin file exists for: 	 210602_GP_JP
Bin file exists for: 	 210603_GP_JP
Converting 2

kilosort.io : ========================================
kilosort.io : ========================================
kilosort.io : Converting 3029 data chunks with a chunksize of 60000 samples...
kilosort.io : Converting 3029 data chunks with a chunksize of 60000 samples...
kilosort.io : 163 of 3029 chunks converted...
kilosort.io : 163 of 3029 chunks converted...
kilosort.io : 309 of 3029 chunks converted...
kilosort.io : 309 of 3029 chunks converted...
kilosort.io : 455 of 3029 chunks converted...
kilosort.io : 455 of 3029 chunks converted...
kilosort.io : 596 of 3029 chunks converted...
kilosort.io : 596 of 3029 chunks converted...
kilosort.io : 734 of 3029 chunks converted...
kilosort.io : 734 of 3029 chunks converted...
kilosort.io : 867 of 3029 chunks converted...
kilosort.io : 867 of 3029 chunks converted...
kilosort.io : 995 of 3029 chunks converted...
kilosort.io : 995 of 3029 chunks converted...
kilosort.io : 1121 of 3029 chunks converted...
kilosort.io : 1121 of 3029 chunks convert

181721280, 18, 1, 30000.0, None
Raw data does not exist for: 	 210609_GP_JP
Raw data does not exist for: 	 210708_GP_JP
Raw data does not exist for: 	 240625_GP_TZ
Raw data does not exist for: 	 240627_GP_TZ
Bin file exists for: 	 240710_GP_TZ


## Run Kilosort

#### KS settings

In [23]:
drift_correction = True

settings_s_probe_ucla = {
    'fs': 30000,
    'batch_size': 120000,
    'min_template_size': 30,
    'nearest_templates': 6,
    'n_chan_bin': 18,
}
settings_s_probe_uw = {
    'fs': 30000,
    'batch_size': 120000,
    'min_template_size': 30,
    'nearest_templates': 6,
    'n_chan_bin': 19,
    'Th_universal': 9,
    'Th_learned': 8,
    
}
if drift_correction:
    settings_s_probe_ucla['nblocks'] = 1
    settings_s_probe_uw['nblocks'] = 1
else:
    settings_s_probe_ucla['nblocks'] = 0
    settings_s_probe_uw['nblocks'] = 0


s_probe = io.load_probe(s_probe_path)
assert s_probe is not None, 'No probe information exported by SpikeInterface'

In [24]:


for idx, session_id in enumerate(session_metadata['session_id']):
    try:
        session_path = Path(sorting_dir, session_id)
        bin_file = f"{str(session_path)}/{session_id}.bin"
        if session_path.exists():
            # importing correct settings and probe
            if session_metadata['probe'][idx] == 's-probe':
                subject_id = session_metadata['subject'][idx]
                if subject_id == 'Jasper':
                    settings = settings_s_probe_ucla
                    probe = s_probe
                elif subject_id == 'Taz':
                    settings = settings_s_probe_uw
                    probe = s_probe
                    
                # run kilosort
                if not Path(session_path,'kilosort4').exists():
                    ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = run_kilosort(
                        settings=settings, probe=probe, filename=bin_file,
                    )
                else:
                    print(f"Already processed with kilosort: {session_id}")
                    
            elif session_metadata['probe'][idx] == 'single':
                print(f"Single electrode probe won't be processed with kilosort: {session_id}")
        else:
            print(f"Session not found: {session_id}")
            continue   

    except Exception as e:
        print(f"Error in {session_id}: {e}")
        continue

kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Sorting Z:\BassoLabShare\Monkey Studies\SC Priors\data\sorting\210608_GP_JP\210608_GP_JP.bin
kilosort.run_kilosort: Sorting Z:\BassoLabShare\Monkey Studies\SC Priors\data\sorting\210608_GP_JP\210608_GP_JP.bin
kilosort.run_kilosort: Sorting Z:\BassoLabShare\Monkey Studies\SC Priors\data\sorting\210608_GP_JP\210608_GP_JP.bin
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary fi

Single electrode probe won't be processed with kilosort: 210126_GP_JP
Session not found: 210205_GP_JP
Session not found: 210210_GP_JP
Session not found: 210211_GP_JP
Session not found: 210216_GP_JP
Session not found: 210217_GP_JP
Session not found: 210224_GP_JP
Session not found: 210302_GP_JP
Session not found: 210305_GP_JP
Session not found: 210311_GP_JP
Session not found: 210312_GP_JP
Session not found: 210315_GP_JP
Session not found: 210316_GP_JP
Session not found: 210317_GP_JP
Session not found: 210318_GP_JP
Session not found: 210323_GP_JP
Session not found: 210413_GP_JP
Session not found: 210524_GP_JP
Session not found: 210525_GP_JP
Session not found: 210528_GP_JP
Session not found: 210601_GP_JP
Already processed with kilosort: 210602_GP_JP
Already processed with kilosort: 210603_GP_JP


kilosort.run_kilosort: Preprocessing filters computed in  8.49s; total  8.50s
kilosort.run_kilosort: Preprocessing filters computed in  8.49s; total  8.50s
kilosort.run_kilosort: Preprocessing filters computed in  8.49s; total  8.50s
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.spikedetect: Re-computing universal templates from data.
kilosort.spikedetect: Re-computing universal templates from data.
kilosort.spikedetect: Re-computing universal templates from data.
100%|██████████| 1515/1515 [18:17<00:00,  1.38it/s]
kilosort.run_kilosort: drift computed in  1165.90s; total  1174.42s
kilosort.run_kilosort: drift

Session not found: 210609_GP_JP
Session not found: 210708_GP_JP
Session not found: 240625_GP_TZ
Session not found: 240627_GP_TZ
Already processed with kilosort: 240710_GP_TZ
